# Imports

In [ ]:
!pip install rake-nltk

     |████████████████████████████████| 1.5 MB 4.3 MB/s 
     |████████████████████████████████| 749 kB 49.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
!pip install pytextrank

     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 1.1 MB 50.4 MB/s 
     |████████████████████████████████| 6.0 MB 38.1 MB/s 
     |████████████████████████████████| 11.2 MB 44.1 MB/s 
     |████████████████████████████████| 930 kB 51.9 MB/s 
     |████████████████████████████████| 10.1 MB 41.8 MB/s 
     |████████████████████████████████| 42 kB 49 kB/s 
     |████████████████████████████████| 457 kB 57.8 MB/s 
     |████████████████████████████████| 653 kB 52.6 MB/s 
     |████████████████████████████████| 181 kB 74.0 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Att

In [ ]:
from google.colab import auth, drive
from google.auth import default
import gspread
import pandas as pd
#from  rake_nltk import Rake  
import nltk
import spacy
#import pytextrank
from collections import Counter
import collections

# Reading Data from Web-Scrapped Set

In [ ]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

def data_retrival(sheet_name):
  worksheet = gc.open(sheet_name).sheet1
  rows = worksheet.get_all_values()
  df = pd.DataFrame.from_records(rows)
  return df

In [ ]:
#Remove useless columns
DS_JOB_DF = data_retrival('DS_job_searched')
DE_JOB_DF = data_retrival('DE_job_searched')

In [ ]:
#DS_JOB_DF.head()
#DE_JOB_DF.head()

# Reading CSV from drive folders

In [ ]:
drive.mount('/content/drive')

#ROOT_DIR = "/content/drive/MyDrive/DTSC630/Projects/Job Screening Dataset" #BH Directory 
ROOT_DIR = "/content/drive/MyDrive/Projects/Job Screening Dataset" #MT Directory 
#ROOT_DIR = #HC DIR
#ROOT_DIR = #BH DIR

GOOG_DATA = ROOT_DIR + '/google_job_skills.csv'
DICE_DATA = ROOT_DIR + '/dice_com-job_us_sample.csv'
KW_DATA = ROOT_DIR + '/Marked_KW_List - Total_Keyword_List.csv'

GOOG_DF_ORIG = pd.read_csv(GOOG_DATA)
DICE_DF_ORIG = pd.read_csv(DICE_DATA)
KW_DF_ORIG = pd.read_csv(KW_DATA)
KW_CAT_ORIG = pd.read_csv(ROOT_DIR + '/Categorized List.csv')


Mounted at /content/drive


In [ ]:
GOOG_DF_ORIG.head()

In [ ]:
DICE_DF_ORIG.head()

# Dataset Preprocessing


## Possible Keyword Methodologies

### NLTK Rake

In [ ]:
#nltk.download('stopwords')
#nltk.download('punkt')
#Impletement spaCy
r = Rake()  
def rake_implement(x,r):
     r.extract_keywords_from_text(x)
     return r.get_ranked_phrases()

## spaCY


In [ ]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 45.7 MB 575 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
#!pip install pytextrank
# Example code from



# example text
text = """Compatibility of systems of linear constraints over the set of natural numbers.
Criteria of compatibility of a system of linear Diophantine equations, strict inequations,
and nonstrict inequations are considered. Upper bounds for components of a minimal set of
solutions and algorithms of construction of minimal generating sets of solutions for all types
of systems are given. These criteria and the corresponding algorithms for constructing a minimal
supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."""

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_md")
# add PyTextRank to the spaCy pipeline
nlp.add_pipe("textrank")

doc = nlp(text)
# examine the top-ranked phrases in the document
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    #print(phrase.chunks)

In [ ]:
def spaCy_keywords(text, nlp):
  key_phrase = []
  doc = nlp(text)
  for phrase in doc._.phrases:
    key = phrase.text.replace('\n', '') 
    key_phrase.append(key)

  return key_phrase


## Web-Scrapped Data

## Google Data

In [ ]:
GOOG_DF_ORIG.describe()

In [ ]:
g_df = GOOG_DF_ORIG[['Category', 'Minimum Qualifications', 'Preferred Qualifications']]
g_df = g_df.apply(lambda x: x.astype(str).str.lower())
g_df.head()

,Category,Minimum Qualifications,Preferred Qualifications
0,program management,ba/bs degree or equivalent practical experienc...,experience in the business technology market a...
1,manufacturing & supply chain,bs degree in an engineering discipline or equi...,"bsee, bsme or bsie degree.\nexperience of usin..."
2,technical solutions,"bachelor’s degree in business, economics, stat...",experience partnering or consulting cross-func...
3,developer relations,ba/bs degree in computer science or equivalent...,"experience as a software developer, architect,..."
4,program management,ba/bs degree or equivalent practical experienc...,cts certification.\nexperience in the construc...


In [ ]:
g_df_min = g_df.groupby(['Category'])['Minimum Qualifications'].apply(','.join).reset_index()
g_df_min.head()


In [ ]:
g_df_min.describe()

In [ ]:
g_df_pref = g_df.groupby(['Category'])['Preferred Qualifications'].apply(','.join).reset_index()
g_df_pref.head()

In [ ]:
g_df_pref.describe()

In [ ]:
g_df['Min Qual KW'] =g_df['Minimum Qualifications'].apply(lambda x: rake_implement(x,r))
#g_df['Min Qual KW'][3]

### Spacy Utilization

In [ ]:
nlp = spacy.load("en_core_web_md") 
nlp.add_pipe("textrank")

In [ ]:
g_df['Min Qual KW'] =g_df['Minimum Qualifications'].apply(lambda x: spaCy_keywords(x,nlp))
g_df['Min Qual KW'][3]

### Performing spaCy per category df creation

In [ ]:
#Planning to do:
#For loop that iterates through list of categories, selecting rows with i category, 
#performing spaCy key word extraction on those rows creating minimum key word list and pref word list
#Utilize counter to find count of key words present in the rows, storing the resulting dictionary in new df
# new df contains category/min qual key words with count dictionary/pref qual key words with count dictionary


In [ ]:
cat_list = g_df['Category'].unique()
total = 0
from collections import Counter
import collections

key_word_df = pd.DataFrame(columns = ['Category', 'Min_KW', 'Pref_KW'])
for cat in cat_list:
  #Current df
  results = Counter()
  curr_df = g_df[g_df['Category']== cat].copy()

  curr_df['Min Qual KW'] =curr_df['Minimum Qualifications'].apply(lambda x: spaCy_keywords(x,nlp)) #Pulls words
  words = curr_df['Min Qual KW'].tolist()
  min_words = collections.Counter([x for sublist in words for x in sublist]) #Frequency count of words

  curr_df['Pref Qual KW'] =curr_df['Preferred Qualifications'].apply(lambda x: spaCy_keywords(x,nlp)) #Pulls words
  words_2 = curr_df['Pref Qual KW'].tolist()
  pref_words = collections.Counter([x for sublist in words_2 for x in sublist]) #Frequency count of words
 
  key_word_df.loc[len(key_word_df.index)] = [cat, dict(min_words), dict(pref_words)]#Load into new df

#Df structure: Cateogry, Dictory of Min Qualifation Words, Dictonary of Pref Qualification words
key_word_df.head()
 
#print(total)

#### Total KeyWord Data Frame Combining Min and Pref QUal

In [ ]:
min_kw = g_df['Minimum Qualifications'].apply(lambda x: spaCy_keywords(x,nlp)).tolist() #Pulls words
pref_kw = g_df['Preferred Qualifications'].apply(lambda x: spaCy_keywords(x,nlp)).tolist() #Pulls words

total_kw = min_kw + pref_kw

word_counter = collections.Counter([x for sublist in total_kw for x in sublist])

kw_df = pd.DataFrame.from_dict(word_counter, orient='index').reset_index()



In [ ]:
kw_df = kw_df.rename(columns={'index':'Key_Word', 0:'Frequency'})
kw_df = kw_df.sort_values('Frequency',ascending=False).reset_index(drop=True)

kw_df.head()

,Key_Word,Frequency
0,experience,2084
1,ability,1306
2,equivalent practical experience,983
3,ba/bs degree,792
4,5 years,396


In [ ]:
kw_df.to_csv(ROOT_DIR+'/Total_Keyword_List.csv')

####Key Word Dataframe, Separated By Category





In [ ]:
key_word_df.describe()

In [ ]:
for indx, row in key_word_df.iterrows():
  print(row['Category'])
  print(len(row['Min_KW']))
  print(len(row['Pref_KW']))

In [ ]:
key_word_df.to_csv(ROOT_DIR+'/Current_Keywords.csv')

In [ ]:
key_word_df = pd.read_csv(ROOT_DIR+'/Current_Keywords.csv')

for indx, row in key_word_df.iterrows():
  print(row['Category'])
  print((row['Min_KW']))


## Key Words Dataframe Preprocessing

In [ ]:
KW_DF_ORIG.head()

,Unnamed: 0,Key_Word,Frequency,Valid
0,0,experience,2084,n
1,1,ability,1306,n
2,2,equivalent practical experience,983,n
3,3,ba/bs degree,792,n
4,4,5 years,396,n


In [ ]:
KW_DF_WORDS = KW_DF_ORIG[KW_DF_ORIG['Valid']== 'y'].reset_index(drop=True)

In [ ]:
KW_DF_WORDS[['Key_Word', 'Frequency']].to_csv(ROOT_DIR + '/KW_FILTERED.csv')

In [ ]:
KW_CAT_ORIG.head()

,Key_Words,Grouping
0,computer science,specialty
1,marketing,specialty
2,consulting,specialty
3,sales,specialty
4,python,tool


## Creating DF with Determing Keywords per category

"bsee, bsme or bsie degree.\nexperience of using statistics tools for data analysis, e.g. distribution histogram/pareto chart, process control chart, design of experiment (doe), correlation analysis, etc.\ndemonstrated knowledge in pcba manufacturing process and quality control.\nfamilar with cable/connector related components' manufacturing: moldling, stamping, die-casting, lim, mim process and materials.\nself starter with innovation, integrity and attention to detail.\nability to travel up to 50% of the timebs degree in an engineering discipline or equivalent practical experience.\n7 years of experience in cable/connector design or manufacturing in an npi role.\nexperience working with interconnect engineering and product design (pd)/mechanical engineer in developing, manufacturing and testing.\nability to speak and write in english and mandarin fluently and idiomatically."

In [ ]:
# Load keyword df
# For loop for each category, where you iterate through kw df and see if any matches occur on the category df
# then get the count of instances for frequency count, finally append the word and count to the dataframe if > 0
cat_list = g_df['Category'].unique()
g_df['Qualifications']= g_df['Preferred Qualifications'] + g_df['Minimum Qualifications']

cat_key_words = pd.DataFrame(columns = ['Category', 'specialty', 'tool', 'trait', 'environment'])

for cat in cat_list:
  spec_list = []
  trait_list = []
  env_list = []
  tool_list = []

  curr_df = g_df[g_df['Category']== cat].copy()
  for index, row in KW_CAT_ORIG.iterrows():
    #print(row['Key_Words'])
    count = 0

    #Note, error with c++ due to the '++' causing error with the contains method
    if row['Key_Words'] == 'c++':
      pass
    else:
      count = len(curr_df[curr_df['Qualifications'].str.contains(str(row['Key_Words']))])

    if count > 0:
      if row['Grouping'] == 'specialty':
        spec_list.append([row['Key_Words'], count])
      elif row['Grouping'] == 'trait':
        trait_list.append([row['Key_Words'], count])

      elif row['Grouping'] == 'tool':
        tool_list.append([row['Key_Words'], count])

      elif row['Grouping'] == 'environment':
        env_list.append([row['Key_Words'], count])

  cat_key_words.loc[len(cat_key_words.index)] = [cat, spec_list, tool_list, trait_list, env_list]#Load into new df



cat_key_words.head()  

  #Select portion of dataframe for this category
  # iterate through dataframe of keywords and see for every word how many jobs match it, 
  #then append to new df


,Category,specialty,tool,trait,environment
0,program management,"[[computer science, 24], [marketing, 9], [cons...","[[python, 5], [java, 5], [javascript, 4], [sql...","[[analytical, 15], [project management, 20], [...","[[changing environment, 1], [team, 32], [fast-..."
1,manufacturing & supply chain,"[[consulting, 1], [mba, 7], [engineering, 15],...","[[r, 16], [erp, 6], [cs, 10], [c, 16]]","[[analytical, 2], [project management, 1], [co...","[[team, 13], [fast-paced environment, 1]]"
2,technical solutions,"[[computer science, 70], [marketing, 13], [con...","[[python, 53], [java, 57], [javascript, 54], [...","[[analytical, 23], [project management, 28], [...","[[changing environment, 1], [team, 56], [fast-..."
3,developer relations,"[[computer science, 5], [engineering, 1], [mac...","[[python, 2], [java, 5], [javascript, 5], [and...",[],[]
4,hardware engineering,"[[computer science, 2], [mba, 1], [engineering...","[[python, 5], [android, 1], [mobile, 1], [matl...","[[project management, 3], [communication, 8], ...","[[team, 8], [mentoring, 1]]"


In [ ]:
cat_key_words

,Category,specialty,tool,trait,environment
0,program management,"[[computer science, 24], [marketing, 9], [cons...","[[python, 5], [java, 5], [javascript, 4], [sql...","[[analytical, 15], [project management, 20], [...","[[changing environment, 1], [team, 32], [fast-..."
1,manufacturing & supply chain,"[[consulting, 1], [mba, 7], [engineering, 15],...","[[r, 16], [erp, 6], [cs, 10], [c, 16]]","[[analytical, 2], [project management, 1], [co...","[[team, 13], [fast-paced environment, 1]]"
2,technical solutions,"[[computer science, 70], [marketing, 13], [con...","[[python, 53], [java, 57], [javascript, 54], [...","[[analytical, 23], [project management, 28], [...","[[changing environment, 1], [team, 56], [fast-..."
3,developer relations,"[[computer science, 5], [engineering, 1], [mac...","[[python, 2], [java, 5], [javascript, 5], [and...",[],[]
4,hardware engineering,"[[computer science, 2], [mba, 1], [engineering...","[[python, 5], [android, 1], [mobile, 1], [matl...","[[project management, 3], [communication, 8], ...","[[team, 8], [mentoring, 1]]"
5,partnerships,"[[computer science, 12], [marketing, 3], [cons...","[[python, 3], [java, 4], [javascript, 1], [sql...","[[analytical, 23], [project management, 14], [...","[[changing environment, 1], [team, 20], [fast-..."
6,product & customer support,"[[computer science, 15], [marketing, 12], [con...","[[python, 12], [java, 9], [javascript, 5], [sq...","[[analytical, 14], [project management, 11], [...","[[team, 24], [fast-paced environment, 3], [cro..."
7,software engineering,"[[computer science, 20], [marketing, 1], [mba,...","[[python, 16], [java, 16], [javascript, 11], [...","[[analytical, 1], [communication, 2], [managem...","[[team, 3], [mentoring, 1]]"
8,data center & network,"[[engineering, 1], [electrical engineering, 1]...","[[r, 2], [c, 2]]","[[project management, 1], [communication, 1], ...",[]
9,business strategy,"[[computer science, 17], [marketing, 14], [con...","[[python, 7], [java, 1], [sql, 24], [iaas, 1],...","[[analytical, 65], [project management, 51], [...","[[changing environment, 35], [team, 22], [fast..."


In [ ]:
cat_key_words.to_csv(ROOT_DIR + '/Categories_With_KeywordLists.csv')

In [ ]:
cat_key_words = pd.read_csv(ROOT_DIR + '/Categories_With_KeywordLists.csv')

cat_key_words.head()


,Unnamed: 0,Category,specialty,tool,trait,environment
0,0,program management,"[['computer science', 24], ['marketing', 9], [...","[['python', 5], ['java', 5], ['javascript', 4]...","[['analytical', 15], ['project management', 20...","[['changing environment', 1], ['team', 32], ['..."
1,1,manufacturing & supply chain,"[['consulting', 1], ['mba', 7], ['engineering'...","[['r', 16], ['erp', 6], ['cs', 10], ['c', 16]]","[['analytical', 2], ['project management', 1],...","[['team', 13], ['fast-paced environment', 1]]"
2,2,technical solutions,"[['computer science', 70], ['marketing', 13], ...","[['python', 53], ['java', 57], ['javascript', ...","[['analytical', 23], ['project management', 28...","[['changing environment', 1], ['team', 56], ['..."
3,3,developer relations,"[['computer science', 5], ['engineering', 1], ...","[['python', 2], ['java', 5], ['javascript', 5]...",[],[]
4,4,hardware engineering,"[['computer science', 2], ['mba', 1], ['engine...","[['python', 5], ['android', 1], ['mobile', 1],...","[['project management', 3], ['communication', ...","[['team', 8], ['mentoring', 1]]"


In [ ]:
cat_list = g_df['Category'].unique()
g_df['Qualifications']= g_df['Preferred Qualifications'] + g_df['Minimum Qualifications']
cat_key_words_2 = pd.DataFrame(columns = ['Category', 'Major','Major_Weights', 'Specialty', 
                                          'Specialty_Weights', 'Tool', 'Tool_Weights', 
                                          'Trait','Trait_Weights', 'Environment', 'Environment_Weights'])

KW_CAT_ORIG = pd.read_csv(ROOT_DIR + '/Categorized List_2.csv')
#print(KW_CAT_ORIG['Grouping'].unique())
for cat in cat_list:
  spec_list = []
  spec_list_2 = []
  spec_list_weight = []
  spec_count = 0

  trait_list = []
  trait_list_2 = []
  trait_list_weight = []
  trait_count = 0

  env_list = []
  env_list_2 = []
  env_list_weight = []
  env_count = 0

  tool_list = []
  tool_list_2 = []
  tool_list_weight = []
  tool_count = 0

  major_list = []
  major_list_2 = []
  major_list_weight = []
  major_count = 0

  curr_df = g_df[g_df['Category']== cat].copy()
  for index, row in KW_CAT_ORIG.iterrows():
    #print(row['Key_Words'])
    count = 0

    #Note, error with c++ due to the '++' causing error with the contains method
    if row['Key_Words'] == 'c++':
      pass
    else:
      count = len(curr_df[curr_df['Qualifications'].str.contains(str(row['Key_Words']))])

    if count > 0:
      if row['Grouping'] == 'specialty':
        spec_list.append([row['Key_Words'], count])
        spec_count += count
      elif row['Grouping'] == 'trait':
        trait_list.append([row['Key_Words'], count])
        trait_count +=count

      elif row['Grouping'] == 'tool':
        tool_list.append([row['Key_Words'], count])
        tool_count +=count

      elif row['Grouping'] == 'environment':
        env_list.append([row['Key_Words'], count])
        env_count +=count

      elif row['Grouping'] == 'major':
        major_list.append([row['Key_Words'], count])
        major_count +=count

  for i in spec_list:
    if spec_count !=0:
      spec_list_2.append(i[0])
      spec_list_weight.append(str(round((i[1]/spec_count),3)))

  for i in trait_list:
    if trait_count != 0:
      trait_list_2.append(i[0])
      trait_list_weight.append(str(round((i[1]/trait_count),3)))

  for i in tool_list:
    if tool_count != 0:
      tool_list_2.append(i[0])
      tool_list_weight.append(str(round((i[1]/tool_count),3)))

  for i in env_list:
    if env_count != 0:
      env_list_2.append(i[0])
      env_list_weight.append(str(round((i[1]/env_count),3)))

  for i in major_list:
    if major_count != 0:
      major_list_2.append(i[0])
      major_list_weight.append(str(round((i[1]/major_count),3)))


  cat_key_words_2.loc[len(cat_key_words_2.index)] = [cat, major_list_2, major_list_weight,
                                                     spec_list_2, spec_list_weight,
                                                     tool_list_2, tool_list_weight,
                                                     trait_list_2, trait_list_weight,
                                                     env_list_2, env_list_weight]#Load into new df



cat_key_words_2.head()  

  #Select portion of dataframe for this category
  # iterate through dataframe of keywords and see for every word how many jobs match it, 
  #then append to new df


,Category,Major,Major_Weights,Specialty,Specialty_Weights,Tool,Tool_Weights,Trait,Trait_Weights,Environment,Environment_Weights
0,program management,"[computer science, marketing, consulting, sale...","[0.118, 0.044, 0.088, 0.103, 0.103, 0.162, 0.0...","[systems engineering, data analysis, technical...","[0.035, 0.07, 0.035, 0.053, 0.018, 0.07, 0.018...","[python, java, javascript, sql, saas, google c...","[0.019, 0.019, 0.015, 0.011, 0.049, 0.034, 0.0...","[analytical, project management, flexibility, ...","[0.072, 0.096, 0.019, 0.048, 0.139, 0.306, 0.0...","[changing environment, team, fast-paced enviro...","[0.024, 0.762, 0.024, 0.048, 0.095, 0.024, 0.024]"
1,manufacturing & supply chain,"[consulting, mba, engineering, statistics, fin...","[0.021, 0.149, 0.319, 0.021, 0.128, 0.128, 0.1...","[data analysis, product design, consumer elect...","[0.03, 0.03, 0.212, 0.424, 0.303]","[r, erp, cs, c]","[0.333, 0.125, 0.208, 0.333]","[analytical, project management, communication...","[0.057, 0.029, 0.314, 0.371, 0.143, 0.029, 0.0...","[team, fast-paced environment]","[0.929, 0.071]"
2,technical solutions,"[computer science, marketing, consulting, sale...","[0.201, 0.037, 0.04, 0.14, 0.029, 0.146, 0.003...","[customer service, big data, systems engineeri...","[0.048, 0.119, 0.123, 0.009, 0.101, 0.048, 0.1...","[python, java, javascript, sql, html, iaas, sa...","[0.054, 0.058, 0.055, 0.028, 0.034, 0.026, 0.0...","[analytical, project management, flexibility, ...","[0.108, 0.131, 0.005, 0.192, 0.315, 0.042, 0.0...","[changing environment, team, fast-paced enviro...","[0.013, 0.7, 0.062, 0.05, 0.175]"
3,developer relations,"[computer science, engineering, science]","[0.455, 0.091, 0.455]",[machine learning],[1.0],"[python, java, javascript, android, mobile, ph...","[0.048, 0.119, 0.119, 0.071, 0.095, 0.024, 0.1...",[],[],[],[]
4,hardware engineering,"[computer science, mba, engineering, finance, ...","[0.045, 0.023, 0.409, 0.023, 0.227, 0.068, 0.0...","[product design, consumer electronics, supply ...","[0.125, 0.375, 0.062, 0.25, 0.125, 0.062]","[python, android, mobile, matlab, r, linux, cs...","[0.064, 0.013, 0.013, 0.013, 0.282, 0.038, 0.1...","[project management, communication, management...","[0.111, 0.296, 0.37, 0.037, 0.111, 0.037, 0.037]","[team, mentoring]","[0.889, 0.111]"


In [ ]:
import csv
cat_key_words_2.to_csv(ROOT_DIR + "/Categories_KW_Normalized_test.csv")


In [ ]:
test = pd.read_csv(ROOT_DIR + "/Categories_KW_Normalized_test.csv")
test.head()

FileNotFoundError: ignored

In [ ]:
for index, row in cat_key_words.iterrows():
  for name, value in row.iteritems():
    count = 0
    if name == 'Unnamed: 0' or name == 'Category':
      continue
    print(type(value))
    df['list'] = pd.eval(df['list'])


<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

## Dice Data

In [ ]:
DICE_DF_ORIG.describe()

In [ ]:
d_df = DICE_DF_ORIG[['jobdescription', 'jobtitle', 'skills']]
d_df = d_df.apply(lambda x: x.astype(str).str.lower())
d_df.head()
d_df.describe()

In [ ]:
d_df[d_df['skills'].str.contains('see')]
